In [1]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
from surprise import Dataset, Reader, KNNWithMeans, SVD, accuracy
from surprise.model_selection import cross_validate, train_test_split


## Reading Data

In [2]:
reviews_train = pd.read_csv('../data/reviews_train.csv', usecols=['RecipeId', 'AuthorId', 'Rating'])
reviews_test = pd.read_csv('../data/reviews_test.csv', usecols=['RecipeId', 'AuthorId', 'Rating'])


In [3]:
reviews_train['Rating'].std()

0.6390110779349543

## Constructing base model

### SVD model

In [4]:
svd = SVD()
reader = Reader(rating_scale=(1,5))
# Loads Pandas dataframe
data = Dataset.load_from_df(reviews_train.loc[:10000, ["AuthorId", "RecipeId", "Rating"]], reader)

In [5]:
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.6498  0.6546  0.6451  0.6534  0.6502  0.6506  0.0033  
MAE (testset)     0.4653  0.4609  0.4526  0.4614  0.4646  0.4610  0.0046  
Fit time          0.15    0.11    0.12    0.10    0.09    0.11    0.02    
Test time         0.01    0.01    0.01    0.01    0.01    0.01    0.00    


{'test_rmse': array([0.64978844, 0.65457466, 0.6451286 , 0.65341172, 0.65023883]),
 'test_mae': array([0.46534321, 0.46093998, 0.45255171, 0.461432  , 0.46464612]),
 'fit_time': (0.14926838874816895,
  0.11138534545898438,
  0.11720657348632812,
  0.10107541084289551,
  0.0922994613647461),
 'test_time': (0.0077610015869140625,
  0.007268428802490234,
  0.0063817501068115234,
  0.007158041000366211,
  0.006520986557006836)}

### KNN model

In [28]:
reader = Reader(rating_scale=(1, 5))
# Loads Pandas dataframe
data_train = Dataset.load_from_df(reviews_train.loc[:10000, ["AuthorId", "RecipeId", "Rating"]], reader)
data_test = Dataset.load_from_df(reviews_test.loc[:100, ["AuthorId", "RecipeId", "Rating"]], reader)

# To use user-based cosine similarity
sim_options = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
trainingSet = data_train.build_full_trainset()
testingSet = data_test.build_full_trainset()
algo = KNNWithMeans(sim_options=sim_options)

In [29]:
# fitting the model
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [49]:
reviews_test[reviews_test['AuthorId']==2946]

,RecipeId,AuthorId,Rating


## Parameter Tuning

In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])